In [ ]:
%pip install -q ultralytics opencv-python
%pip install -q pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
import os

# 2. Load video
video_path = "../utilities/cars_on_the_highway.mp4"
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file {video_path} not found. Please check the path.")

# 3. Load YOLO model for detection
model_path = "../utilities/yolov8n.pt"  # Path to your YOLO model
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file {model_path} not found. Please check the path.")

model = YOLO(model_path)

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

print(f"Detected video resolution: {frame_width}x{frame_height}")

# # 4. Tracking futtatása beépített API-val
results = model.track(
    source=video_path,
    conf=0.25,  # lowered it from 0.4, to see if there would be more entries in the csv
    iou=0.5,
    show=False,           # Ha True: megjelenítés futás közben
    save=True,            # Mentett videó a 'runs/track' mappába
    save_txt=True,        # Ment ".txt" formátumú detekciós adatokat
    classes=[2, 5, 7],     # car, bus, truck
    tracker="botsort.yaml"  # vagy "bytetrack.yaml", "botsort.yaml", stb.
)

# 5. Detekciós eredmények TXT-ből CSV-be alakítása
# Fájlok: runs/track/predict/labels/frame_x.txt

labels_dir = "runs/detect/track/labels"
tracked_data = []

for file in sorted(os.listdir(labels_dir)):
    if not file.endswith(".txt"):
        continue
    # Pl.: "cars_on_the_highway_7.txt" → frame_idx = 7
    frame_idx = int(file.rsplit("_", 1)[1].split(".")[0])
    with open(os.path.join(labels_dir, file), "r") as f:
        for line in f.readlines():
            obj = line.strip().split()
            if len(obj) != 6:
                print(f"Fájl kihagyva, hibás sor: {line}")
                continue
            class_id = int(obj[0])
            x_center = float(obj[1])
            y_center = float(obj[2])
            width = float(obj[3])
            height = float(obj[4])
            track_id = int(obj[5])

            # YOLO-format -> pixel coords (valódi videóméret alapján)
            x1 = int((x_center - width / 2) * frame_width)
            y1 = int((y_center - height / 2) * frame_height)
            x2 = int((x_center + width / 2) * frame_width)
            y2 = int((y_center + height / 2) * frame_height)

            tracked_data.append([frame_idx, track_id, x1, y1, x2, y2])

# 6. Eredmények mentése CSV-be
columns = ["frame", "id", "x1", "y1", "x2", "y2"]
df = pd.DataFrame(tracked_data, columns=columns)
df.to_csv("trajectory_data_yolov8.csv", index=False)
print("Tracking data saved to trajectory_data_yolov8.csv")


Detected video resolution: 1920x1080
Újabb sor hozzáadva: 1
Újabb sor hozzáadva: 1
Újabb sor hozzáadva: 24
Újabb sor hozzáadva: 24
Újabb sor hozzáadva: 24
Fájl kihagyva, hibás sor: 2 0.602038 0.436262 0.024552 0.0348126

Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 24
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 24
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 24
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb sor hozzáadva: 133
Újabb s